In [1]:
import time
import tdmclient.notebook
import numpy as np

In [ ]:
# ==========================================================
# 1. LOCAL NAVIGATION MODULE (Logic & State)
# ==========================================================

# Constants
THRESH_ENTRY = 2000      
THRESH_WALL_LOST = 400   
TARGET_DIST = 2000       
BASE_SPEED = 100
P_GAIN = 0.1

# Times
CLEARANCE_DURATION = 1 
TURN_DURATION = 1.2    

# Debug
DEBUG_PRINT = True

# State variables (Global scope in Notebook)
current_state = "GLOBAL"
timer_start = 0
previous_wall_side = None 

# --- Logic Functions ---

def _clamp(val):
    return max(min(int(val), 500), -500)

def reset_local_state():
    global current_state, timer_start, previous_wall_side
    current_state = "GLOBAL"
    timer_start = 0
    previous_wall_side = None

def check_obstacle_trigger(prox_horizontal):
    front_all = prox_horizontal[0:5]
    if max(front_all) > THRESH_ENTRY:
        return True
    return False

def local_nav_log(message):
    if DEBUG_PRINT:
        print(f"[LOCAL_NAV] {message}")

def local_nav_update(prox_horizontal):
    """
    Pure logic: takes sensors, returns speed (left, right).
    """
    global current_state, timer_start, previous_wall_side
    
    # sensors
    front_all = prox_horizontal[0:5]
    front_center = prox_horizontal[2]
    left_side = prox_horizontal[0]
    right_side = prox_horizontal[4]
    
    # STATE 1: GLOBAL
    if current_state == "GLOBAL":
        if max(front_all) > THRESH_ENTRY:
            local_nav_log("Obstacle detected. Choosing side.")
            sum_left = prox_horizontal[0] + prox_horizontal[1] 
            sum_right = prox_horizontal[3] + prox_horizontal[4]
            
            if sum_left > (sum_right + 50):
                current_state = "FOLLOW_LEFT"
                previous_wall_side = "LEFT"
            else:
                current_state = "FOLLOW_RIGHT"
                previous_wall_side = "RIGHT"
            return 0, 0
        return BASE_SPEED, BASE_SPEED

    # STATE 2: FOLLOW LEFT
    elif current_state == "FOLLOW_LEFT":
        if front_center > THRESH_ENTRY:
            return 100, -100 
        if left_side < THRESH_WALL_LOST:
            local_nav_log("Wall finished. Clearance.")
            current_state = "CLEARANCE"
            timer_start = time.time()
            return BASE_SPEED, BASE_SPEED
        error = left_side - TARGET_DIST
        correction = int(error * P_GAIN)
        return _clamp(BASE_SPEED + correction), _clamp(BASE_SPEED - correction)

    # STATE 3: FOLLOW RIGHT
    elif current_state == "FOLLOW_RIGHT":
        if front_center > THRESH_ENTRY:
            return -100, 100 
        if right_side < THRESH_WALL_LOST:
            local_nav_log("Wall finished. Clearance.")
            current_state = "CLEARANCE"
            timer_start = time.time()
            return BASE_SPEED, BASE_SPEED
        error = right_side - TARGET_DIST
        correction = int(error * P_GAIN)
        return _clamp(BASE_SPEED - correction), _clamp(BASE_SPEED + correction)

    # STATE 4: CLEARANCE
    elif current_state == "CLEARANCE":

        if max(front_all) > THRESH_ENTRY:
            current_state = f"FOLLOW_{previous_wall_side}"
            return 0, 0
        if (time.time() - timer_start) >= CLEARANCE_DURATION:
            local_nav_log("Realigning.")
            current_state = "REALIGN_ANGLE"
            timer_start = time.time()
            return 0, 0
        return BASE_SPEED, BASE_SPEED

    # STATE 5: REALIGN
    elif current_state == "REALIGN_ANGLE":
        if (time.time() - timer_start) >= TURN_DURATION:
            if max(front_all) > THRESH_ENTRY:
                local_nav_log("Obstacle still present.")
                current_state = "GLOBAL" 
            else:
                local_nav_log("Path clear. Global.")
                current_state = "GLOBAL"
            return 0, 0
        
        if previous_wall_side == "LEFT":
            return -100, 100 
        else:
            return 100, -100 

    return 0, 0

In [3]:
# ==========================================================
# 2. HARDWARE HELPERS (Synchronous)
# ==========================================================

# NOTE: In tdmclient notebook mode, we modify global variables directly.
# The @sync_var decorator ensures they are sent/received from the robot.

@tdmclient.notebook.sync_var
def set_motors(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = int(left)
    motor_right_target = int(right)

@tdmclient.notebook.sync_var
def set_lights_on():
    global leds_top
    leds_top = [0, 0, 32] # Blue

@tdmclient.notebook.sync_var
def set_lights_off():
    global leds_top
    leds_top = [0, 0, 0] # Off

@tdmclient.notebook.sync_var
def stop_robot():
    global motor_left_target, motor_right_target
    motor_left_target = 0
    motor_right_target = 0

@tdmclient.notebook.sync_var
def get_prox_sensors():
    global prox_horizontal
    return prox_horizontal

In [ ]:
# Import CV modules if needed
import cv2
import time
from CV_aruco_module import get_pose_from_frame, draw_global_path
from global_nav_module import get_global_path

# 1. Start the Notebook Session
await tdmclient.notebook.start()

# State initialization
state = "ROTATE"
thymio_start = thymio_theta = goal = polygons = homography = None
global_path = None

def enter_local_navigation():
    global state
    reset_local_state()
    state = "LOCAL_NAVIGATION"

print("Thymio connected.")

try:
    # cap = cv2.VideoCapture(0)

    while True:
        # 1. Throttle loop (Standard sleep)
        time.sleep(0.1)

        # 2. Read Sensors (Synchronous call)
        prox_values = get_prox_sensors()
        
        # Debug Print
        # print(f"Max Front Sensor: {max(prox_values[:5])}")

        # ==================== STATE MACHINE ====================
        
        if state == "GLOBAL_NAVIGATION":
            print("State: GLOBAL NAV")
            # Global nav logic here...
            stop_robot()
        
        elif state == "ROTATE":
            print("State: ROTATE")
            
            # Check Obstacle
            if check_obstacle_trigger(prox_values):
                local_nav_log("Obstacle detected! Switching to Local.")
                enter_local_navigation()
                set_lights_on()
                continue 
            
            # Move (Rotate)
            set_motors(100, 100) # Change to (100, -100) to actually rotate

        elif state == "FORWARD":
            print("State: FORWARD")
            
            # Check Obstacle
            if check_obstacle_trigger(prox_values):
                local_nav_log("Obstacle detected! Switching to Local.")
                stop_robot()
                enter_local_navigation()
                set_lights_on()
                continue 
            
            # Move (Forward)
            set_motors(100, 100)

        elif state == "LOCAL_NAVIGATION":
            print("State: LOCAL NAV")
            
            # Logic
            left_speed, right_speed = local_nav_update(prox_values)
            
            # Act
            set_motors(left_speed, right_speed)

            # Exit Condition
            if current_state == "GLOBAL" and max(prox_values[:5]) < THRESH_ENTRY:
                local_nav_log("Obstacle cleared. Returning to Global.")
                stop_robot()
                set_lights_off()
                state = "GLOBAL_NAVIGATION"

        # CV / Visualization Logic
        # ret, frame = cap.read()
        # if ret:
        #    cv2.imshow("Feed", frame)
        #    if cv2.waitKey(1) & 0xFF == ord('q'):
        #        break

except KeyboardInterrupt:
    print("Stopping...")

finally:
    # Cleanup
    stop_robot()
    set_lights_off()
    # cap.release()
    # cv2.destroyAllWindows()
    print("Shutdown complete")

Thymio connected.
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
State: ROTATE
[LOCAL_NAV] Obstacle detected! Switching to Local.
State: LOCAL NAV
[LOCAL_NAV] Obstacle detected. Choosing side.
State: LOCAL NAV
[LOCAL_NAV] Wall finished. Clearance.
State: LOCAL NAV
State: LOCAL NAV
[LOCAL_NAV] Wall finished. Clearance.
State: LOCAL NAV
State: LOCAL NAV
[LOCAL_NAV] Wall finished. Clearance.
State: LOCAL NAV
State: LOCAL NAV
[LOCAL_NAV] Wall finished. Clearance.
State: LOCAL NAV
State: LOCAL NAV
[LOCAL_NAV] Wall finished. Clearance.
State: LOCAL NAV
State: LOCAL NAV
[LOCAL_NAV] Wall finished. Clearance.
State: LOCAL NAV
State: LOCAL NAV

In [ ]:
import tdmclient.notebook
await tdmclient.notebook.stop()

ValueError: Function <function _pre_run_cell at 0x000001F79738E340> is not registered as a pre_run_cell callback